In [11]:
# from google.colab import files
# uploaded = files.upload()

In [12]:
from pandas import DataFrame
import pandas

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pandas.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [13]:
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

dataset = read_csv('data/pm2.5_non_normalised.csv', header=0, index_col=0)

In [14]:
import copy
import os
from math import sqrt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
import numpy as np

def grid_search(dataset, lookback, nodes, layers, split_percentage):
    scores = list()
  
    for lookback_period in lookback:
        values = series_to_supervised(dataset, lookback_period, 1).values

        n_train_hours = (int)(split_percentage * len(values))
        train = values[:n_train_hours, :]
        test = values[n_train_hours:, :]

        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], lookback_period, 1))
        test_X = test_X.reshape((test_X.shape[0], lookback_period, 1))

        for number_of_nodes in nodes:            
            for number_of_layers in layers:
                model = Sequential()
                for i in range(number_of_layers):
                    if i == 0 :
                        model.add(LSTM(number_of_nodes, return_sequences=True, input_shape=(train_X.shape[1], 1)))
                    elif i == number_of_layers - 1 :
                        model.add(LSTM(number_of_nodes))
                    else :
                        model.add(LSTM(number_of_nodes, return_sequences=True))

                model.add(Dense(1))
                model.compile(loss='mae', optimizer='adam')


                name = str(lookback_period)+'_'+str(number_of_nodes)+'_'+str(number_of_layers)
                print("Starting training for : ", name)

                history = model.fit(train_X, train_y, epochs=100, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
                yhat = model.predict(test_X)

                np.save('LSTM_runtime/history' + name, history)
                np.save('LSTM_runtime/test_y' + name, test_y)
                np.save('LSTM_runtime/yhat' + name, yhat)
                model.save('LSTM_runtime/model/' + name + '.h5')

                rmse = mean_squared_error(yhat, test_y)
                print("RMSE for ", name, " = " , rmse)

                scores.append([rmse, lookback_period, number_of_nodes, number_of_layers])
                np.save('LSTM_scores', np.array(scores))

    return scores
          
  

In [15]:
layers = [2,4,8]
lookback = [1,3,5,7]
nodes = [25,50,75]
split_percentage = 0.8

scores = grid_search(dataset, lookback, nodes, layers, split_percentage)
np.save('LSTM_scores', np.array(scores))

W1101 19:58:59.903560 139871435638592 deprecation_wrapper.py:119] From /home/acs-lab/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1101 19:58:59.917372 139871435638592 deprecation_wrapper.py:119] From /home/acs-lab/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1101 19:58:59.920503 139871435638592 deprecation_wrapper.py:119] From /home/acs-lab/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1101 19:59:00.300484 139871435638592 deprecation_wrapper.py:119] From /home/acs-lab/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Starting training for :  1_25_2


W1101 19:59:00.621810 139871435638592 deprecation.py:323] From /home/acs-lab/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1101 19:59:01.741853 139871435638592 deprecation_wrapper.py:119] From /home/acs-lab/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1101 19:59:01.861703 139871435638592 deprecation_wrapper.py:119] From /home/acs-lab/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 35038 samples, validate on 8760 samples
Epoch 1/100
 - 3s - loss: 85.3357 - val_loss: 81.6778
Epoch 2/100
 - 2s - loss: 75.4885 - val_loss: 74.8290
Epoch 3/100
 - 2s - loss: 69.5498 - val_loss: 69.1206
Epoch 4/100
 - 2s - loss: 64.4704 - val_loss: 64.1164
Epoch 5/100
 - 2s - loss: 60.0025 - val_loss: 59.6619
Epoch 6/100
 - 2s - loss: 56.0487 - val_loss: 55.7464
Epoch 7/100
 - 2s - loss: 52.4774 - val_loss: 52.2328
Epoch 8/100
 - 2s - loss: 49.2884 - val_loss: 49.0747
Epoch 9/100
 - 2s - loss: 46.4618 - val_loss: 46.2618
Epoch 10/100
 - 2s - loss: 43.9338 - val_loss: 43.7588
Epoch 11/100
 - 2s - loss: 41.6502 - val_loss: 41.5217
Epoch 12/100
 - 2s - loss: 39.5925 - val_loss: 39.5254
Epoch 13/100
 - 2s - loss: 37.7469 - val_loss: 37.7428
Epoch 14/100
 - 2s - loss: 36.0412 - val_loss: 36.1282
Epoch 15/100
 - 2s - loss: 34.4897 - val_loss: 34.6300
Epoch 16/100
 - 2s - loss: 33.0789 - val_loss: 33.3172
Epoch 17/100
 - 1s - loss: 31.7926 - val_loss: 32.1699
Epoch 18/100
 - 1s - loss

 - 3s - loss: 17.9496 - val_loss: 18.2126
Epoch 48/100
 - 3s - loss: 17.7874 - val_loss: 18.1527
Epoch 49/100
 - 3s - loss: 17.6458 - val_loss: 18.0299
Epoch 50/100
 - 3s - loss: 17.4867 - val_loss: 17.7240
Epoch 51/100
 - 3s - loss: 17.3944 - val_loss: 17.7077
Epoch 52/100
 - 3s - loss: 17.2790 - val_loss: 17.3966
Epoch 53/100
 - 3s - loss: 17.1755 - val_loss: 17.3891
Epoch 54/100
 - 3s - loss: 17.0817 - val_loss: 17.2253
Epoch 55/100
 - 3s - loss: 16.9832 - val_loss: 17.3897
Epoch 56/100
 - 3s - loss: 16.9158 - val_loss: 17.0265
Epoch 57/100
 - 3s - loss: 16.8453 - val_loss: 16.8680
Epoch 58/100
 - 3s - loss: 16.7159 - val_loss: 16.7696
Epoch 59/100
 - 3s - loss: 16.6312 - val_loss: 16.8513
Epoch 60/100
 - 3s - loss: 16.5683 - val_loss: 16.8868
Epoch 61/100
 - 3s - loss: 16.5167 - val_loss: 16.7712
Epoch 62/100
 - 3s - loss: 16.4641 - val_loss: 16.4190
Epoch 63/100
 - 3s - loss: 16.3576 - val_loss: 16.3364
Epoch 64/100
 - 3s - loss: 16.2629 - val_loss: 16.1529
Epoch 65/100
 - 3s - lo

...

Epoch 12/100
 - 20s - loss: 35.7257 - val_loss: 35.5256
Epoch 13/100
 - 20s - loss: 33.8111 - val_loss: 33.8112
Epoch 14/100
 - 20s - loss: 32.0827 - val_loss: 32.1794
Epoch 15/100
 - 20s - loss: 30.5609 - val_loss: 30.7028
Epoch 16/100
 - 19s - loss: 29.1597 - val_loss: 29.4239
Epoch 17/100
 - 20s - loss: 27.9440 - val_loss: 28.2572
Epoch 18/100
 - 20s - loss: 26.8908 - val_loss: 27.3431
Epoch 19/100
 - 20s - loss: 25.8382 - val_loss: 26.5417
Epoch 20/100
 - 20s - loss: 24.9844 - val_loss: 25.8545
Epoch 21/100
 - 20s - loss: 24.1989 - val_loss: 24.9602
Epoch 22/100
 - 19s - loss: 23.5255 - val_loss: 24.2233
Epoch 23/100
 - 20s - loss: 22.8627 - val_loss: 23.7981
Epoch 24/100
 - 20s - loss: 22.2716 - val_loss: 23.0834
Epoch 25/100
 - 20s - loss: 21.7396 - val_loss: 22.4139
Epoch 26/100
 - 20s - loss: 21.3435 - val_loss: 22.0900
Epoch 27/100
 - 20s - loss: 20.8211 - val_loss: 21.5216
Epoch 28/100
 - 20s - loss: 20.4101 - val_loss: 20.9663
Epoch 29/100
 - 20s - loss: 20.0277 - val_loss: 

Epoch 58/100
 - 7s - loss: 14.1271 - val_loss: 13.2176
Epoch 59/100
 - 7s - loss: 14.0614 - val_loss: 13.2656
Epoch 60/100
 - 7s - loss: 14.1023 - val_loss: 13.0908
Epoch 61/100
 - 7s - loss: 14.0356 - val_loss: 13.5757
Epoch 62/100
 - 7s - loss: 14.0286 - val_loss: 13.1395
Epoch 63/100
 - 7s - loss: 13.9891 - val_loss: 13.1810
Epoch 64/100
 - 7s - loss: 14.0701 - val_loss: 13.4297
Epoch 65/100
 - 7s - loss: 14.0043 - val_loss: 13.1291
Epoch 66/100
 - 7s - loss: 13.9699 - val_loss: 13.3129
Epoch 67/100
 - 7s - loss: 13.8543 - val_loss: 13.0976
Epoch 68/100
 - 7s - loss: 13.8736 - val_loss: 13.0817
Epoch 69/100
 - 7s - loss: 13.8645 - val_loss: 13.0464
Epoch 70/100
 - 7s - loss: 13.8716 - val_loss: 13.0917
Epoch 71/100
 - 7s - loss: 13.8099 - val_loss: 13.0722
Epoch 72/100
 - 7s - loss: 13.8213 - val_loss: 13.1785
Epoch 73/100
 - 7s - loss: 13.7575 - val_loss: 13.0989
Epoch 74/100
 - 7s - loss: 13.7701 - val_loss: 13.1023
Epoch 75/100
 - 7s - loss: 13.8402 - val_loss: 12.9446
Epoch 76/1

 - 38s - loss: 79.5438 - val_loss: 75.8249
Epoch 2/100
 - 20s - loss: 71.2889 - val_loss: 70.5503
Epoch 3/100
 - 20s - loss: 66.6328 - val_loss: 67.7867
Epoch 4/100
 - 20s - loss: 63.3431 - val_loss: 58.4591
Epoch 5/100
 - 20s - loss: 52.0253 - val_loss: 48.9977
Epoch 6/100
 - 20s - loss: 44.8140 - val_loss: 42.6425
Epoch 7/100
 - 20s - loss: 39.5809 - val_loss: 38.1969
Epoch 8/100
 - 20s - loss: 35.6024 - val_loss: 34.9365
Epoch 9/100
 - 20s - loss: 32.3907 - val_loss: 31.9953
Epoch 10/100
 - 20s - loss: 29.7518 - val_loss: 29.7887
Epoch 11/100
 - 20s - loss: 27.5487 - val_loss: 27.7371
Epoch 12/100
 - 20s - loss: 25.7551 - val_loss: 26.0063
Epoch 13/100
 - 20s - loss: 24.3192 - val_loss: 24.6361
Epoch 14/100
 - 20s - loss: 23.1392 - val_loss: 23.4621
Epoch 15/100
 - 20s - loss: 22.0549 - val_loss: 22.5006
Epoch 16/100
 - 20s - loss: 21.1454 - val_loss: 21.6295
Epoch 17/100
 - 20s - loss: 20.4353 - val_loss: 20.7972
Epoch 18/100
 - 20s - loss: 19.7388 - val_loss: 20.1308
Epoch 19/100


Epoch 47/100
 - 7s - loss: 13.9135 - val_loss: 12.9260
Epoch 48/100
 - 8s - loss: 13.9443 - val_loss: 13.3093
Epoch 49/100
 - 7s - loss: 13.9356 - val_loss: 13.3736
Epoch 50/100
 - 8s - loss: 13.9128 - val_loss: 13.0752
Epoch 51/100
 - 7s - loss: 13.8584 - val_loss: 13.1964
Epoch 52/100
 - 7s - loss: 13.8357 - val_loss: 13.1777
Epoch 53/100
 - 7s - loss: 13.8485 - val_loss: 13.3329
Epoch 54/100
 - 7s - loss: 13.8647 - val_loss: 13.4639
Epoch 55/100
 - 7s - loss: 13.7907 - val_loss: 13.0758
Epoch 56/100
 - 7s - loss: 13.8578 - val_loss: 13.0545
Epoch 57/100
 - 7s - loss: 13.7514 - val_loss: 13.0420
Epoch 58/100
 - 7s - loss: 13.7913 - val_loss: 12.9361
Epoch 59/100
 - 7s - loss: 13.7936 - val_loss: 13.4034
Epoch 60/100
 - 7s - loss: 13.7604 - val_loss: 13.2988
Epoch 61/100
 - 7s - loss: 13.7688 - val_loss: 13.8033
Epoch 62/100
 - 7s - loss: 13.6732 - val_loss: 13.3031
Epoch 63/100
 - 7s - loss: 13.7196 - val_loss: 12.7942
Epoch 64/100
 - 8s - loss: 13.6704 - val_loss: 12.9055
Epoch 65/1

Epoch 93/100
 - 12s - loss: 13.3802 - val_loss: 13.6819
Epoch 94/100
 - 12s - loss: 13.6221 - val_loss: 13.8106
Epoch 95/100
 - 12s - loss: 13.2914 - val_loss: 13.6932
Epoch 96/100
 - 12s - loss: 13.5563 - val_loss: 13.0092
Epoch 97/100
 - 12s - loss: 13.3517 - val_loss: 13.5768
Epoch 98/100
 - 12s - loss: 13.3753 - val_loss: 14.0934
Epoch 99/100
 - 12s - loss: 13.2600 - val_loss: 14.5320
Epoch 100/100
 - 12s - loss: 13.4778 - val_loss: 13.5272
RMSE for  7_75_4  =  680.9869197649921
Starting training for :  7_75_8
Train on 35033 samples, validate on 8759 samples
Epoch 1/100
 - 40s - loss: 77.0592 - val_loss: 72.7703
Epoch 2/100
 - 21s - loss: 69.3233 - val_loss: 68.2741
Epoch 3/100
 - 21s - loss: 63.9933 - val_loss: 54.4077
Epoch 4/100
 - 21s - loss: 47.7787 - val_loss: 43.9004
Epoch 5/100
 - 21s - loss: 39.6401 - val_loss: 37.2919
Epoch 6/100
 - 21s - loss: 34.3024 - val_loss: 33.0311
Epoch 7/100
 - 21s - loss: 30.4092 - val_loss: 29.5789
Epoch 8/100
 - 21s - loss: 27.4684 - val_loss:

In [16]:
print(scores)

[[883.286548232861, 1, 25, 2], [908.8345276511375, 1, 25, 4], [948.3146706725935, 1, 25, 8], [712.6354280027632, 1, 50, 2], [715.7250441131793, 1, 50, 4], [715.7641344996393, 1, 50, 8], [685.8534798587331, 1, 75, 2], [693.2918327153462, 1, 75, 4], [702.0623337224893, 1, 75, 8], [728.199864595387, 3, 25, 2], [767.6470038989008, 3, 25, 4], [751.1694889522232, 3, 25, 8], [659.8595320767549, 3, 50, 2], [648.74443035603, 3, 50, 4], [667.1733063768002, 3, 50, 8], [651.5987569185187, 3, 75, 2], [612.6065142692344, 3, 75, 4], [640.4156848829284, 3, 75, 8], [715.6605981117416, 5, 25, 2], [752.2984722017788, 5, 25, 4], [778.3012376439058, 5, 25, 8], [648.2640424384806, 5, 50, 2], [684.7767005758616, 5, 50, 4], [704.6737962116481, 5, 50, 8], [625.6513037012052, 5, 75, 2], [682.2350440526244, 5, 75, 4], [624.0085444699617, 5, 75, 8], [767.1502539679376, 7, 25, 2], [758.8086772694134, 7, 25, 4], [782.8342035574884, 7, 25, 8], [680.7056597440736, 7, 50, 2], [695.7336254466934, 7, 50, 4], [690.100985